In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from skfolio import RiskMeasure
from skfolio.optimization import MeanRisk

In [47]:
tickers = ["AAPL", "GOOGL", "JNJ", "JPM", "MSFT"]

stocks = yf.download(tickers, start='2020-01-01', end='2025-12-31', auto_adjust=False)
stocks_adj_stocks = stocks["Adj Close"]
stocks_adj_stocks = stocks_adj_stocks.dropna()

# Resetear el índice para convertir 'Date' en una columna normal
stocks_adj_stocks = stocks_adj_stocks.reset_index()
stocks_adj_stocks

[*********************100%***********************]  5 of 5 completed


Ticker,Date,AAPL,GOOGL,JNJ,JPM,MSFT
0,2020-01-02,72.716057,68.108376,126.055176,120.733566,153.323257
1,2020-01-03,72.009117,67.752075,124.595741,119.140335,151.414154
2,2020-01-06,72.582901,69.557945,124.440300,119.045593,151.805496
3,2020-01-07,72.241547,69.423592,125.200218,117.021744,150.421356
4,2020-01-08,73.403641,69.917732,125.182961,117.934639,152.817322
...,...,...,...,...,...,...
1326,2025-04-11,198.149994,157.139999,151.729996,236.199997,388.450012
1327,2025-04-14,202.520004,159.070007,154.360001,234.720001,387.809998
1328,2025-04-15,202.139999,156.309998,153.619995,233.130005,385.730011
1329,2025-04-16,194.270004,153.330002,153.910004,229.610001,371.609985


In [ ]:
import matplotlib.pyplot as plt

# Asegurarse de que la columna 'Date' es de tipo datetime (por si acaso)
stocks_adj_stocks['Date'] = pd.to_datetime(stocks_adj_stocks['Date'])

# Establecer 'Date' como índice temporal para facilitar el plot
stocks_adj_stocks.set_index('Date', inplace=True)

# Graficar
plt.figure(figsize=(14, 7))
for ticker in stocks_adj_stocks.columns:
    plt.plot(stocks_adj_stocks.index, stocks_adj_stocks[ticker], label=ticker)

plt.title("Precios ajustados de acciones en 2013")
plt.xlabel("Fecha")
plt.ylabel("Precio ajustado ($)")
plt.legend(loc="upper left", fontsize="small")
plt.grid(True)
plt.tight_layout()
plt.show()



## Ejercicio 1: Portafolio de mínima varianza
    • Descripción: Encuentra el portafolio de mínima varianza utilizando un conjunto de retornos históricos de 5 activos.
    • Pasos: 
        1. Carga o genera datos de retornos históricos para 5 activos.
        2. Usa skfolio para estimar la matriz de covarianzas de los retornos.
        3. Optimiza el portafolio para minimizar la varianza (riesgo) sin considerar los retornos esperados.

Muestra los pesos óptimos de cada activo.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from skfolio import RiskMeasure
from skfolio.optimization import MeanRisk


# Calcula retornos logarítmicos diarios
returns = np.log(stocks_adj_stocks / stocks_adj_stocks.shift(1)).dropna()

# Paso 2: Estimar la matriz de covarianzas
# Calcular matriz de covarianzas empírica (anualizada)
cov_matrix = returns.cov() * 252  # Anualizar

# Visualizar la matriz de covarianzas como un mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cov_matrix, annot=True, fmt=".6f", cmap="coolwarm", 
            xticklabels=returns.columns, yticklabels=returns.columns, cbar_kws={'label': 'Covarianza'})
plt.title("Matriz de Covarianzas (Anualizada)")
plt.show()

# Paso 3: Optimizar el portafolio para mínima varianza
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,  # Minimizar varianza
    portfolio_params=dict(name="Minimum Variance Portfolio")
)

# Ajustar el modelo
model.fit(returns)

# Extraer los pesos (usar predict para obtener el portafolio)
portfolio = model.predict(returns)
weights = portfolio.weights

# Paso 4: Mostrar los pesos óptimos
weights_dict = dict(zip(returns.columns, weights))
print("\nPesos óptimos del portafolio de mínima varianza:")
for ticker, weight in weights_dict.items():
    print(f"{ticker}: {weight:.4f}")

In [54]:
returns_pct = stocks_adj_stocks.pct_change().dropna()

returns_ln = np.log(stocks_adj_stocks / stocks_adj_stocks.shift(1)).dropna()

print(f"Los returns con pct_change son : {returns_pct}")

print(f"Los returns con log nepariano son : {returns_ln}")

Los returns con pct_change son : Ticker          AAPL     GOOGL       JNJ       JPM      MSFT
Date                                                        
2020-01-03 -0.009722 -0.005231 -0.011578 -0.013196 -0.012451
2020-01-06  0.007968  0.026654 -0.001248 -0.000795  0.002585
2020-01-07 -0.004703 -0.001932  0.006107 -0.017001 -0.009118
2020-01-08  0.016086  0.007118 -0.000138  0.007801  0.015928
2020-01-09  0.021241  0.010498  0.002966  0.003651  0.012493
...              ...       ...       ...       ...       ...
2025-04-11  0.040594  0.028268  0.020445  0.040025  0.018618
2025-04-14  0.022054  0.012282  0.017333 -0.006266 -0.001648
2025-04-15 -0.001876 -0.017351 -0.004794 -0.006774 -0.005363
2025-04-16 -0.038933 -0.019065  0.001888 -0.015099 -0.036606
2025-04-17  0.013950 -0.014152  0.023130  0.010235 -0.010306

[1330 rows x 5 columns]
Los returns con log nepariano son : Ticker          AAPL     GOOGL       JNJ       JPM      MSFT
Date                                                

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from matplotlib.ticker import FuncFormatter

returns = stocks_adj_stocks.pct_change().dropna()

# Matriz de covarianzas anualizada
cov_matrix = returns.cov() * 252

# Pesos del portafolio de mínima varianza (los que obtuviste)
weights_min_var = np.array([0.0068, 0.1160, 0.7291, 0.0678, 0.0803])

# Pesos del portafolio equiponderado
weights_equal = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Paso 1: Calcular la varianza del portafolio de mínima varianza
variance_min_var = np.dot(weights_min_var.T, np.dot(cov_matrix, weights_min_var))
print(f"Varianza anualizada del portafolio de mínima varianza: {variance_min_var:.6f}")

# Calcular la varianza del portafolio equiponderado
variance_equal = np.dot(weights_equal.T, np.dot(cov_matrix, weights_equal))
print(f"Varianza anualizada del portafolio equiponderado: {variance_equal:.6f}")

# Paso 2: Calcular los retornos diarios de ambos portafolios
portfolio_returns_min_var = returns.dot(weights_min_var)  # Retornos del portafolio de mínima varianza
portfolio_returns_equal = returns.dot(weights_equal)      # Retornos del portafolio equiponderado

# Paso 3: Calcular el valor acumulado de la inversión
cumulative_returns_min_var = (1 + portfolio_returns_min_var).cumprod()
cumulative_returns_equal = (1 + portfolio_returns_equal).cumprod()

# Paso 4: Calcular la rentabilidad acumulada en porcentaje
rentabilidad_acumulada_min_var = (cumulative_returns_min_var - 1) * 100
rentabilidad_acumulada_equal = (cumulative_returns_equal - 1) * 100

# Configurar el estilo visual para un diseño más profesional
plt.style.use('seaborn-v0_8-whitegrid')

# Crear el gráfico con dimensiones óptimas
fig, ax = plt.subplots(figsize=(14, 8), dpi=100)

# Colores mejorados
color_min_var = '#4C72B0'  # Azul elegante para mínima varianza
color_equal = 'black'      # Negro para equiponderado como solicitado

# Graficar las series con mayor grosor y mejor diseño
ax.plot(rentabilidad_acumulada_min_var, 
        label='Portafolio de Mínima Varianza', 
        linewidth=2.5, 
        color=color_min_var)

ax.plot(rentabilidad_acumulada_equal, 
        label='Portafolio Equiponderado', 
        linewidth=1, 
        color=color_equal,
        linestyle='--')  # Línea discontinua para el portafolio equiponderado


# Mejorar la apariencia del gráfico
ax.set_title('Comparación de Rentabilidad Acumulada: Mínima Varianza vs Equiponderado', 
             fontsize=22, 
             fontweight='bold', 
             pad=20)

ax.set_xlabel('Fecha', fontsize=18, fontweight='medium', labelpad=10)
ax.set_ylabel('Rentabilidad Acumulada (%)', fontsize=18, fontweight='medium', labelpad=10)

# Formatear el eje Y para mostrar porcentajes correctamente
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.1f}%'))

# Mejorar la leyenda
ax.legend(fontsize=14, 
          loc='best', 
          frameon=True, 
          framealpha=0.9, 
          edgecolor='lightgray')

# Ajustar bordes y espaciado
plt.tight_layout()

# Añadir grid sutil
ax.grid(True, linestyle='--', alpha=0.6)

# Mejorar márgenes y ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Guardar la figura en alta calidad si se desea
# plt.savefig('comparacion_min_var_vs_equal.png', dpi=300, bbox_inches='tight')

# Mostrar el gráfico
plt.show()

### Manera cuántica

In [58]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
import numpy as np

# Parámetros
n_assets = 5
K = 2
lambda_ = 1000  # Penalización
mu = 1000       # Penalización
Sigma = np.array([
    [0.106848, 0.069817, 0.022697, 0.046125, 0.074243],
    [0.069817, 0.108024, 0.017160, 0.044433, 0.074636],
    [0.022697, 0.017160, 0.039843, 0.025230, 0.020561],
    [0.046125, 0.044433, 0.025230, 0.106234, 0.043248],
    [0.074243, 0.074636, 0.020561, 0.043248, 0.093748]
])

# Crear programa cuadrático
qp = QuadraticProgram()

# Añadir variables binarias: 5 activos x 5 niveles = 25 variables
for i in range(n_assets):
    for k in range(K + 1):
        qp.binary_var(f"x_{i}_{k}")

# Término de varianza
quadratic = {}
for i in range(n_assets):
    for j in range(n_assets):
        for k in range(K + 1):
            for l in range(K + 1):
                coeff = (k / 4) * (l / 4) * Sigma[i, j]
                quadratic[(f"x_{i}_{k}", f"x_{j}_{l}")] = quadratic.get((f"x_{i}_{k}", f"x_{j}_{l}"), 0) + coeff

# Penalización por suma de pesos = 1
linear = {}
for i in range(n_assets):
    for k in range(K + 1):
        coeff = lambda_ * ( (k / 4) ** 2 - 2 * (k / 4) )
        linear[f"x_{i}_{k}"] = linear.get(f"x_{i}_{k}", 0) + coeff
for i in range(n_assets):
    for j in range(i + 1, n_assets):
        for k in range(K + 1):
            for l in range(K + 1):
                coeff = 2 * lambda_ * (k / 4) * (l / 4)
                quadratic[(f"x_{i}_{k}", f"x_{j}_{l}")] = quadratic.get((f"x_{i}_{k}", f"x_{j}_{l}"), 0) + coeff

# Penalización por un nivel por activo
for i in range(n_assets):
    for k in range(K + 1):
        linear[f"x_{i}_{k}"] = linear.get(f"x_{i}_{k}", 0) - mu
    for k in range(K + 1):
        for l in range(k + 1, K + 1):
            quadratic[(f"x_{i}_{k}", f"x_{i}_{l}")] = quadratic.get((f"x_{i}_{k}", f"x_{i}_{l}"), 0) + 2 * mu

# Definir objetivo
qp.minimize(linear=linear, quadratic=quadratic)

# Resolver con QAOA usando el optimizador COBYLA
optimizer = COBYLA()
qaoa = QAOA(sampler=Sampler(), optimizer=optimizer, reps=1)
optimizer = MinimumEigenOptimizer(qaoa)
result = optimizer.solve(qp)

# Mostrar pesos óptimos
weights = np.zeros(n_assets)
for i in range(n_assets):
    for k in range(K + 1):
        if result.x[i * (K + 1) + k] == 1:
            weights[i] = k / 4

print("Pesos óptimos:", dict(zip(["AAPL", "GOOGL", "JNJ", "JPM", "MSFT"], weights)))

C:\Users\gdelvallea001\AppData\Local\Temp\ipykernel_6924\900019327.py:64: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=optimizer, reps=1)


Pesos óptimos: {'AAPL': 0.0, 'GOOGL': 0.0, 'JNJ': 0.5, 'JPM': 0.25, 'MSFT': 0.25}


## Ejercicio 2: Optimización con restricciones de pesos
    • Descripción: Optimiza el portafolio minimizando la varianza, pero con la restricción de que ningún activo puede tener un peso mayor al 20%.

In [103]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from skfolio import RiskMeasure
from skfolio.optimization import MeanRisk

# Paso 3: Optimizar el portafolio para mínima varianza
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,  # Minimizar varianza
    max_weights=0.30,
    min_weights=0,
    portfolio_params=dict(name="Minimum Variance Portfolio With Weight Restriction")
)

# Ajustar el modelo
model.fit(returns)

# Extraer los pesos (usar predict para obtener el portafolio)
portfolio = model.predict(returns)
weights = portfolio.weights

# Paso 4: Mostrar los pesos óptimos
weights_dict = dict(zip(returns.columns, weights))
print("\nPesos óptimos del portafolio de mínima varianza con restricción de pesos:")
for ticker, weight in weights_dict.items():
    print(f"{ticker}: {weight:.4f}")


Pesos óptimos del portafolio de mínima varianza con restricción de pesos:
AAPL: 0.0954
GOOGL: 0.1521
JNJ: 0.3000
JPM: 0.2522
MSFT: 0.2003


In [ ]:
# Convertir los retornos logarítmicos a retornos simples
returns = stocks_adj_stocks.pct_change().dropna()

# Matriz de covarianzas anualizada
cov_matrix = returns.cov() * 252

# Definir los pesos de ambos portafolios
weights_min_var = {
    'AAPL': 0.0068,
    'GOOGL': 0.1160,
    'JNJ': 0.7291,
    'JPM': 0.0678,
    'MSFT': 0.0803
}

weights_constrained = {
    'AAPL': 0.0948,
    'GOOGL': 0.1499,
    'JNJ': 0.3000,
    'JPM': 0.2529,
    'MSFT': 0.2024
}


# Crear arrays de pesos en el mismo orden que las columnas de los retornos
tickers = returns.columns
w_min_var = np.array([weights_min_var[t] for t in tickers])
w_constrained = np.array([weights_constrained[t] for t in tickers])
weights_equal = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Calcular la varianza del portafolio de mínima varianza
variance_min_var = np.dot(w_min_var.T, np.dot(cov_matrix, w_min_var))
print(f"Varianza anualizada del portafolio de mínima varianza: {variance_min_var:.6f}")

# Calcular la varianza del portafolio equiponderado
variance_equal = np.dot(weights_equal.T, np.dot(cov_matrix, weights_equal))
print(f"Varianza anualizada del portafolio equiponderado: {variance_equal:.6f}")

# Calcular la varianza del portafolio de mínima varianza con restricción 
variance_min_var_constrained = np.dot(w_constrained.T, np.dot(cov_matrix, w_constrained))
print(f"Varianza anualizada del portafolio de mínima varianza con restricción: {variance_min_var_constrained:.6f}")

# Calcular retornos diarios de cada portafolio
portfolio_min_var = (returns * w_min_var).sum(axis=1)
portfolio_constrained = (returns * w_constrained).sum(axis=1)
portfolio_returns_equal = returns.dot(weights_equal)

# Calcular la evolución acumulada de la inversión
cumulative_min_var = (1 + portfolio_min_var).cumprod()
cumulative_constrained = (1 + portfolio_constrained).cumprod()
cumulative_returns_equal = (1 + portfolio_returns_equal).cumprod()

# Calcular la rentabilidad acumulada en porcentaje
rentabilidad_acumulada_min_var = (cumulative_min_var - 1) * 100
rentabilidad_acumulada_constrained = (cumulative_constrained - 1) * 100
rentabilidad_acumulada_equal = (cumulative_returns_equal - 1) * 100

# Configurar el estilo visual para un diseño más profesional
plt.style.use('seaborn-v0_8-whitegrid')

# Crear el gráfico con dimensiones óptimas
fig, ax = plt.subplots(figsize=(14, 8), dpi=100)

# Colores más atractivos (negro para mínima varianza, un color elegante para el otro)
color_equal = 'black'  # Negro para mínima varianza
color_constrained = '#4C72B0'  # Azul elegante
color_min_var = '#C44E52' 

# Graficar las series con mayor grosor y mejor diseño
ax.plot(rentabilidad_acumulada_min_var, 
        label='Portafolio de Mínima Varianza', 
        linewidth=1, 
        color=color_min_var)  # Línea discontinua como solicitado

ax.plot(rentabilidad_acumulada_constrained, 
        label='Portafolio con Restricciones', 
        linewidth=2.5, 
        color=color_constrained)

ax.plot(rentabilidad_acumulada_equal, 
        label='Portafolio Equiponderado', 
        linewidth=1, 
        color=color_equal,
        linestyle='--')

# Mejorar la apariencia del gráfico
ax.set_title('Comparaciones de Rentabilidad Acumulada Mínima Varianza', 
             fontsize=22, 
             fontweight='bold', 
             pad=20)

ax.set_xlabel('Fecha', fontsize=18, fontweight='medium', labelpad=10)
ax.set_ylabel('Rentabilidad Acumulada (%)', fontsize=18, fontweight='medium', labelpad=10)

# Formatear el eje Y para mostrar porcentajes correctamente
from matplotlib.ticker import FuncFormatter
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.1f}%'))

# Mejorar la leyenda
ax.legend(fontsize=14, 
          loc='best', 
          frameon=True, 
          framealpha=0.9, 
          edgecolor='lightgray')

# Ajustar bordes y espaciado
plt.tight_layout()

# Añadir grid sutil
ax.grid(True, linestyle='-', alpha=0.6)

# Mejorar márgenes y ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Mostrar el gráfico
plt.show()

In [ ]:
import skfolio
print(skfolio.__version__)
# Veamos qué parámetros acepta realmente MeanRisk
from skfolio.optimization import MeanRisk
help(MeanRisk)
# O inspeccionar la firma del constructor
import inspect
print(inspect.signature(MeanRisk.__init__))

## Ejercicio 3: Optimización multiobjetivo
    • Descripción: Optimiza un portafolio considerando retorno y riesgo simultáneamente.
    • Pasos: 
        1. Define una función objetivo que combine retorno esperado, varianza.
        2. Usa skfolio para optimizar esta función multiobjetivo.
        3. Ajusta los pesos de los objetivos.
Muestra los pesos óptimos resultantes.

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
from plotly.io import show
from sklearn.model_selection import train_test_split

# Importar skfolio y herramientas asociadas
from skfolio import PerfMeasure, RatioMeasure, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import MeanRisk
from skfolio.preprocessing import prices_to_returns

# Cargar el conjunto de datos del S&P 500
# Este conjunto contiene los precios diarios de los 20 activos más importantes del índice S&P 500
precios = load_sp500_dataset()

# Convertir los precios en rendimientos
# Esto convierte los precios de cada activo en su variación porcentual diaria
X = prices_to_returns(precios)

# Dividir los datos en un conjunto de entrenamiento (66%) y uno de prueba (33%)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

# Crear el modelo de optimización de carteras con riesgo medido por la varianza
# Esto optimiza la cartera buscando 30 carteras sobre la frontera eficiente
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,   # Usamos la varianza como medida de riesgo
    efficient_frontier_size=30,           # Encontramos 30 carteras sobre la frontera eficiente
    portfolio_params=dict(name="Variance"),  # Nombre del modelo
)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train)

# Ver las ponderaciones de los activos en las 30 carteras obtenidas
print(model.weights_.shape)  # Esto debe mostrar (30, 20), es decir, 30 carteras y 20 activos

# Predecir las composiciones de las carteras para el conjunto de entrenamiento
population_train = model.predict(X_train)

# Predecir las composiciones de las carteras para el conjunto de prueba
population_test = model.predict(X_test)

# Etiquetar las carteras como "Train" (entrenamiento) y "Test" (prueba) para diferenciarlas
population_train.set_portfolio_params(tag="Train")
population_test.set_portfolio_params(tag="Test")

# Concatenar las poblaciones de carteras de entrenamiento y prueba
population = population_train + population_test

# Visualización: Graficar las carteras con:
# - Eje X: Desviación estándar anualizada (riesgo)
# - Eje Y: Rentabilidad anualizada (rendimiento)
# - Color: Índice de Sharpe anualizado (relación riesgo-retorno)
fig = population.plot_measures(
    x=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION,   # Medir el riesgo usando la desviación estándar anualizada
    y=PerfMeasure.ANNUALIZED_MEAN,                # Medir el rendimiento usando la rentabilidad anualizada
    color_scale=RatioMeasure.ANNUALIZED_SHARPE_RATIO,  # Color según el índice de Sharpe
    hover_measures=[RiskMeasure.MAX_DRAWDOWN, RatioMeasure.ANNUALIZED_SORTINO_RATIO],  # Medidas al pasar el ratón
)

# Mostrar el gráfico
show(fig)

# Graficar la composición de las carteras de entrenamiento (qué activos componen cada cartera)
population_train.plot_composition()

# Imprimir los valores del índice de Sharpe de las carteras de prueba
# El índice de Sharpe mide el rendimiento ajustado por riesgo (un valor más alto es mejor)
print(population_test.measures(measure=RatioMeasure.ANNUALIZED_SHARPE_RATIO))

# Ahora, vamos a ajustar el modelo para encontrar carteras con restricciones de rentabilidad mínima.
# Vamos a buscar carteras que minimicen la varianza bajo las siguientes restricciones de rentabilidad mínima (anualizadas):
# - 15%, 20%, 25%, 30% y 35%
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,  # Medir el riesgo con varianza
    min_return=np.array([0.15, 0.20, 0.25, 0.30, 0.35]) / 252,  # Rentabilidad mínima, dividiendo entre 252 para convertirla a diaria
    portfolio_params=dict(name="Variance"),  # Nombre del modelo
)

# Ajustar el modelo y predecir las carteras
population = model.fit_predict(X_train)

# Graficar las carteras optimizadas con las restricciones de rentabilidad mínima
fig = population.plot_measures(
    x=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION,   # Eje X: Riesgo
    y=PerfMeasure.ANNUALIZED_MEAN,                # Eje Y: Rentabilidad
    color_scale=RatioMeasure.ANNUALIZED_SHARPE_RATIO,  # Color según el índice de Sharpe
    hover_measures=[RiskMeasure.MAX_DRAWDOWN, RatioMeasure.ANNUALIZED_SORTINO_RATIO],  # Medidas adicionales al pasar el ratón
)

# Mostrar el gráfico con las carteras optimizadas
show(fig)



In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from skfolio import PerfMeasure, RatioMeasure, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import MeanRisk
from skfolio.preprocessing import prices_to_returns

# Cargar el conjunto de datos del S&P 500
precios = load_sp500_dataset()

# Convertir los precios en rendimientos
X = prices_to_returns(precios)

# Dividir los datos en entrenamiento y prueba
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

# Crear el modelo de optimización con más puntos en la frontera eficiente
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    efficient_frontier_size=30,  # Aumentamos a 30 carteras para una curva más suave
    portfolio_params=dict(name="Variance"),
    risk_free_rate=0.0416  
)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train)

# Predecir las composiciones de las carteras para el conjunto de entrenamiento
population_train = model.predict(X_train)

# Extraer las medidas de riesgo y rentabilidad para graficar manualmente
risks = population_train.measures(measure=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION)
returns = population_train.measures(measure=PerfMeasure.ANNUALIZED_MEAN)
sharpe_ratios = population_train.measures(measure=RatioMeasure.ANNUALIZED_SHARPE_RATIO)

# Encontrar el índice del máximo Sharpe Ratio
max_sharpe_idx = np.argmax(sharpe_ratios)

# Crear un gráfico con Plotly que conecte los puntos
fig = go.Figure()

# Agregar la curva de la frontera eficiente
fig.add_trace(
    go.Scatter(
        x=risks,
        y=returns,
        mode="lines+markers",  # Conectar los puntos con líneas y mostrar marcadores
        marker=dict(
            color=sharpe_ratios,  # Colorear según el índice de Sharpe
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Sharpe Ratio"),
        ),
        text=[f"Sharpe: {s:.2f}" for s in sharpe_ratios],
        hoverinfo="x+y+text",
    )
)

# Destacar el punto con máximo Sharpe en rojo
fig.add_trace(
    go.Scatter(
        x=[risks[max_sharpe_idx]],
        y=[returns[max_sharpe_idx]],
        mode="markers",
        marker=dict(
            color="red",
            size=10,
            line=dict(width=1, color="DarkSlateGrey")
        ),
        text=[f"""<b>MÁXIMO SHARPE</b><br>"""],
        hoverinfo="x+y+text",
        name="Máximo Sharpe Ratio"
    )
)

# Configurar los ejes y el título
fig.update_layout(
    title="Frontera Eficiente",
    xaxis_title="Desviación Estándar Anualizada",
    yaxis_title="Rentabilidad Anualizada",
    showlegend=False,
)

# Mostrar el gráfico
fig.show()

# Generar el gráfico de composición
fig = population_train.plot_composition()
fig.show()

# Obtener los pesos de todas las carteras
weights_all = np.array([portfolio.weights for portfolio in population_train])

# Calcular los retornos diarios de todas las carteras usando X_test
retornos_diarios_all = X_test.dot(weights_all.T)

# Calcular la rentabilidad acumulada
cumulative_returns = (1 + retornos_diarios_all).cumprod(axis=0) - 1

# Convertir a porcentajes
cumulative_returns_pct = cumulative_returns * 100

# Configurar el estilo visual inspirado en tu gráfico
plt.style.use('seaborn-v0_8-whitegrid')

# Crear el gráfico con dimensiones óptimas
fig, ax = plt.subplots(figsize=(14, 8), dpi=100)

# Graficar todas las carteras en gris, excepto la de máximo Sharpe
for i in range(len(population_train)):
    if i != max_sharpe_idx:
        ax.plot(
            cumulative_returns_pct.iloc[:, i],
            color='grey',
            linewidth=0.5,
            alpha=0.3,
            label='_nolegend_'  # Evita que aparezca en la leyenda
        )

# Graficar la cartera con máximo Sharpe en rojo
ax.plot(
    cumulative_returns_pct.iloc[:, max_sharpe_idx],
    color='#C44E52',  # Rojo similar al de tu gráfico
    linewidth=2.5,
    label='Máximo Sharpe Ratio'
)

# Mejorar la apariencia del gráfico
ax.set_title('Rentabilidad Acumulada de las Carteras', 
             fontsize=22, 
             fontweight='bold', 
             pad=20)

ax.set_xlabel('Fecha', fontsize=18, fontweight='medium', labelpad=10)
ax.set_ylabel('Rentabilidad Acumulada (%)', fontsize=18, fontweight='medium', labelpad=10)

# Formatear el eje Y para mostrar porcentajes
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.1f}%'))

# Mejorar la leyenda
ax.legend(fontsize=14, 
          loc='best', 
          frameon=True, 
          framealpha=0.9, 
          edgecolor='lightgray')

# Añadir grid sutil
ax.grid(True, linestyle='-', alpha=0.6)

# Mejorar márgenes y ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Ajustar bordes y espaciado
plt.tight_layout()


In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split
from skfolio import PerfMeasure, RatioMeasure, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import MeanRisk
from skfolio.preprocessing import prices_to_returns

# Configurar el tema de Plotly para un aspecto profesional
pio.templates.default = "plotly_white"

# Cargar el conjunto de datos del S&P 500
precios = load_sp500_dataset()

# Convertir los precios en rendimientos
X = prices_to_returns(precios)

# Dividir los datos en entrenamiento y prueba
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

# Crear el modelo de optimización con más puntos en la frontera eficiente
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    efficient_frontier_size=50,  # Aumentamos a 50 carteras para una curva más suave
    portfolio_params=dict(name="Variance"),
    risk_free_rate=0.0416  
)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train)

# Predecir las composiciones de las carteras para el conjunto de entrenamiento
population_train = model.predict(X_train)

# Extraer las medidas de riesgo y rentabilidad para graficar manualmente
risks = population_train.measures(measure=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION)
returns = population_train.measures(measure=PerfMeasure.ANNUALIZED_MEAN)
sharpe_ratios = population_train.measures(measure=RatioMeasure.ANNUALIZED_SHARPE_RATIO)

# Encontrar el índice del máximo Sharpe Ratio
max_sharpe_idx = np.argmax(sharpe_ratios)

# --------------- GRÁFICO 1: FRONTERA EFICIENTE MEJORADA ---------------
fig = go.Figure()

# Agregar la curva de la frontera eficiente con gradiente de color mejorado
fig.add_trace(
    go.Scatter(
        x=risks,
        y=returns,
        mode="lines+markers",
        marker=dict(
            size=8,
            color=sharpe_ratios,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(
                title=dict(
                    text="Ratio de Sharpe",
                    font=dict(size=14, family="Arial, sans-serif")
                ),
                tickfont=dict(size=12, family="Arial, sans-serif")
            ),
            line=dict(width=1, color="white")
        ),
        line=dict(color='rgba(100, 110, 250, 0.6)', width=2),
        text=[f"Sharpe: {s:.3f}<br>Riesgo: {r:.2%}<br>Retorno: {ret:.2%}" 
              for s, r, ret in zip(sharpe_ratios, risks, returns)],
        hovertemplate="<b>Cartera</b><br>%{text}<extra></extra>",
    )
)

# Destacar el punto con máximo Sharpe en rojo
fig.add_trace(
    go.Scatter(
        x=[risks[max_sharpe_idx]],
        y=[returns[max_sharpe_idx]],
        mode="markers",
        marker=dict(
            symbol="star",
            color="#E50914",  # Rojo brillante
            size=16,
            line=dict(width=2, color="white")
        ),
        text=[f"<b>MÁXIMO SHARPE: {sharpe_ratios[max_sharpe_idx]:.3f}</b><br>Riesgo: {risks[max_sharpe_idx]:.2%}<br>Retorno: {returns[max_sharpe_idx]:.2%}"],
        hovertemplate="%{text}<extra></extra>",
        name="Máximo Sharpe Ratio"
    )
)

# Configurar los ejes y el título con mejor diseño
fig.update_layout(
    title=dict(
        text="Frontera Eficiente de Carteras",
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    xaxis=dict(
        title=dict(
            text="Desviación Estándar Anualizada",
            font=dict(size=16, family="Arial, sans-serif")
        ),
        tickformat=".1%",
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=False
    ),
    yaxis=dict(
        title=dict(
            text="Rentabilidad Anualizada",
            font=dict(size=16, family="Arial, sans-serif")
        ),
        tickformat=".1%",
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=False
    ),
    showlegend=True,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(211, 211, 211, 0.8)'
    ),
    plot_bgcolor='white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    shapes=[
        # Línea para indicar el rendimiento libre de riesgo
        dict(
            type="line",
            xref="x",
            yref="y",
            x0=0,
            y0=0.0416,  # Tasa libre de riesgo
            x1=risks[max_sharpe_idx],
            y1=returns[max_sharpe_idx],
            line=dict(
                color="rgba(0, 150, 136, 0.7)",
                width=2,
                dash="dash",
            ),
        )
    ],
    annotations=[
        dict(
            x=risks[max_sharpe_idx]/2,
            y=(returns[max_sharpe_idx] + 0.0416)/2,
            text="Línea de Mercado de Capitales",
            showarrow=True,
            arrowhead=2,
            arrowcolor="rgba(0, 150, 136, 0.7)",
            ax=40,
            ay=-40,
            font=dict(
                family="Arial, sans-serif",
                size=12,
                color="rgba(0, 150, 136, 1)"
            ),
        )
    ]
)

# Mostrar el gráfico
fig.show()

# --------------- GRÁFICO 2: COMPOSICIÓN DE CARTERAS MEJORADO ---------------
# Crear un gráfico de composición personalizado más estético
fig_comp = go.Figure()

# Obtener los pesos de todas las carteras
weights_all = np.array([portfolio.weights for portfolio in population_train])

# Obtener los nombres de los activos
asset_names = X_train.columns.tolist()

# Crear una paleta de colores más atractiva
colors = plt.cm.tab20(np.linspace(0, 1, len(asset_names)))
colors = [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})' for r, g, b, _ in colors]

# Ordenar por riesgo (eje x)
sort_indices = np.argsort(risks)
sorted_risks = risks[sort_indices]
sorted_weights = weights_all[sort_indices]

# Posiciones para el eje x
x = np.arange(len(sorted_risks))

# Agregar cada activo como una capa apilada
bottom = np.zeros(len(sorted_risks))
for i, asset in enumerate(asset_names):
    asset_weights = sorted_weights[:, i]
    fig_comp.add_trace(
        go.Bar(
            x=x,
            y=asset_weights,
            name=asset,
            marker_color=colors[i % len(colors)],
            hovertemplate=f"{asset}: %{{y:.2%}}<extra></extra>",
        )
    )

# Marcar la cartera con máximo Sharpe
max_sharpe_pos = np.where(sort_indices == max_sharpe_idx)[0][0]
fig_comp.add_trace(
    go.Scatter(
        x=[max_sharpe_pos],
        y=[1.05],
        mode="markers+text",
        marker=dict(
            symbol="triangle-down",
            color="#E50914",
            size=16
        ),
        text=["Máximo Sharpe"],
        textposition="top center",
        hoverinfo="none",
        showlegend=False
    )
)

# Configurar el diseño del gráfico
fig_comp.update_layout(
    title=dict(
        text="Composición de las Carteras de la Frontera Eficiente",
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    xaxis=dict(
        title="Carteras (ordenadas por riesgo creciente)",
        tickvals=[],
        showgrid=False
    ),
    yaxis=dict(
        title="Ponderación en la Cartera",
        tickformat=".0%",
        range=[0, 1.1],
        gridcolor='rgba(211, 211, 211, 0.6)'
    ),
    barmode='stack',
    plot_bgcolor='white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    )
)

# Mostrar el gráfico
fig_comp.show()

# --------------- GRÁFICO 3: RENDIMIENTO ACUMULADO MEJORADO ---------------
# Obtener los pesos de todas las carteras
weights_all = np.array([portfolio.weights for portfolio in population_train])

# Calcular los retornos diarios de todas las carteras usando X_test
retornos_diarios_all = X_test.dot(weights_all.T)

# Calcular la rentabilidad acumulada
cumulative_returns = (1 + retornos_diarios_all).cumprod(axis=0) - 1

# Crear el gráfico con plotly para mejor interactividad
fig_returns = go.Figure()

# Agregar todas las carteras en gris claro
for i in range(len(population_train)):
    if i != max_sharpe_idx:
        fig_returns.add_trace(
            go.Scatter(
                x=cumulative_returns.index,
                y=cumulative_returns.iloc[:, i],
                mode='lines',
                line=dict(color='rgba(200, 200, 200, 0.3)', width=1),
                showlegend=False,
                hoverinfo='skip'
            )
        )

# Agregar la cartera de máximo Sharpe en rojo brillante
fig_returns.add_trace(
    go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.iloc[:, max_sharpe_idx],
        mode='lines',
        line=dict(color='#E50914', width=3),
        name='Cartera de Máximo Sharpe',
        hovertemplate='<b>%{x}</b><br>Rendimiento: %{y:.2%}<extra></extra>'
    )
)

# Mejorar el diseño del gráfico
fig_returns.update_layout(
    title=dict(
        text='Rendimiento Acumulado de las Carteras en Período de Prueba',
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    xaxis=dict(
        title='Fecha',
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=False
    ),
    yaxis=dict(
        title='Rendimiento Acumulado',
        tickformat='.0%',
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=True,
        zerolinecolor='rgba(0, 0, 0, 0.2)',
        zerolinewidth=1
    ),
    plot_bgcolor='white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    legend=dict(
        y=0.99,
        x=0.01,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(211, 211, 211, 0.8)'
    ),
    hovermode='x unified'
)

# Resaltar la región donde la cartera de máximo Sharpe tiene mejor rendimiento
final_returns = cumulative_returns.iloc[-1, :]
max_final_return = final_returns.max()
max_final_return_idx = final_returns.argmax()

# Agregar anotación para destacar el mejor rendimiento
if max_final_return_idx == max_sharpe_idx:
    best_text = "¡La cartera de máximo Sharpe logró el mejor rendimiento!"
else:
    best_text = f"Cartera de máximo Sharpe: {final_returns[max_sharpe_idx]:.2%} de rendimiento"

fig_returns.add_annotation(
    x=cumulative_returns.index[-1],
    y=cumulative_returns.iloc[-1, max_sharpe_idx],
    text=best_text,
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowcolor="#E50914",
    ax=-150,
    ay=-40,
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="#E50914",
    font=dict(color="#E50914")
)

# Mostrar el gráfico
fig_returns.show()

# --------------- GRÁFICO 4: ESTADÍSTICAS DE LA CARTERA ÓPTIMA ---------------
# Crear un dashboard con las estadísticas clave de la cartera óptima
top_assets_idx = np.argsort(weights_all[max_sharpe_idx])[::-1][:10]  # Top 10 activos
top_assets = [asset_names[i] for i in top_assets_idx]
top_weights = [weights_all[max_sharpe_idx][i] for i in top_assets_idx]

# Crear un gráfico de pastel para los componentes principales
fig_stats = go.Figure()

fig_stats.add_trace(
    go.Pie(
        labels=top_assets,
        values=top_weights,
        hole=0.4,
        textinfo='label+percent',
        marker=dict(
            colors=plt.cm.Paired(np.linspace(0, 1, len(top_assets))),
            line=dict(color='white', width=2)
        ),
        textfont=dict(size=14),
        insidetextorientation='radial'
    )
)

# Configurar el diseño
fig_stats.update_layout(
    title=dict(
        text="Composición de la Cartera Óptima (Máximo Sharpe)",
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    annotations=[
        dict(
            text=f"Sharpe: {sharpe_ratios[max_sharpe_idx]:.3f}<br>Riesgo: {risks[max_sharpe_idx]:.2%}<br>Retorno: {returns[max_sharpe_idx]:.2%}",
            x=0.5,
            y=0.5,
            font=dict(size=16, family="Arial, sans-serif", color="#E50914"),
            showarrow=False
        )
    ],
    width=800,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    )
)

fig_stats.show()

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split
from skfolio import PerfMeasure, RatioMeasure, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import MeanRisk
from skfolio.preprocessing import prices_to_returns

# Configurar el tema de Plotly para un aspecto profesional
pio.templates.default = "plotly_white"

# Cargar el conjunto de datos del S&P 500
precios = load_sp500_dataset()

# Convertir los precios en rendimientos
X = prices_to_returns(precios)

# Crear el modelo de optimización con más puntos en la frontera eficiente
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    efficient_frontier_size=50,  # Aumentamos a 50 carteras para una curva más suave
    portfolio_params=dict(name="Variance"),
    risk_free_rate=0.0416  
)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X)

# Predecir las composiciones de las carteras para el conjunto de entrenamiento
population_train = model.predict(X)

# Extraer las medidas de riesgo y rentabilidad para graficar manualmente
risks = population_train.measures(measure=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION)
returns = population_train.measures(measure=PerfMeasure.ANNUALIZED_MEAN)
sharpe_ratios = population_train.measures(measure=RatioMeasure.ANNUALIZED_SHARPE_RATIO)

# Encontrar el índice del máximo Sharpe Ratio
max_sharpe_idx = np.argmax(sharpe_ratios)

# --------------- GRÁFICO 1: FRONTERA EFICIENTE MEJORADA ---------------
fig = go.Figure()

# Agregar la curva de la frontera eficiente con gradiente de color mejorado
fig.add_trace(
    go.Scatter(
        x=risks,
        y=returns,
        mode="lines+markers",
        marker=dict(
            size=8,
            color=sharpe_ratios,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(
                title=dict(
                    text="Ratio de Sharpe",
                    font=dict(size=14, family="Arial, sans-serif")
                ),
                tickfont=dict(size=12, family="Arial, sans-serif")
            ),
            line=dict(width=1, color="white")
        ),
        line=dict(color='rgba(100, 110, 250, 0.6)', width=2),
        text=[f"Sharpe: {s:.3f}<br>Riesgo: {r:.2%}<br>Retorno: {ret:.2%}" 
              for s, r, ret in zip(sharpe_ratios, risks, returns)],
        hovertemplate="<b>Cartera</b><br>%{text}<extra></extra>",
    )
)

# Destacar el punto con máximo Sharpe en rojo
fig.add_trace(
    go.Scatter(
        x=[risks[max_sharpe_idx]],
        y=[returns[max_sharpe_idx]],
        mode="markers",
        marker=dict(
            symbol="star",
            color="#E50914",  # Rojo brillante
            size=16,
            line=dict(width=2, color="white")
        ),
        text=[f"<b>MÁXIMO SHARPE: {sharpe_ratios[max_sharpe_idx]:.3f}</b><br>Riesgo: {risks[max_sharpe_idx]:.2%}<br>Retorno: {returns[max_sharpe_idx]:.2%}"],
        hovertemplate="%{text}<extra></extra>",
        name="Máximo Sharpe Ratio"
    )
)

# Configurar los ejes y el título con mejor diseño
fig.update_layout(
    title=dict(
        text="Frontera Eficiente de Carteras",
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    xaxis=dict(
        title=dict(
            text="Desviación Estándar Anualizada",
            font=dict(size=16, family="Arial, sans-serif")
        ),
        tickformat=".1%",
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=False
    ),
    yaxis=dict(
        title=dict(
            text="Rentabilidad Anualizada",
            font=dict(size=16, family="Arial, sans-serif")
        ),
        tickformat=".1%",
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=False
    ),
    showlegend=True,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(211, 211, 211, 0.8)'
    ),
    plot_bgcolor='white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    shapes=[
        # Línea para indicar el rendimiento libre de riesgo
        dict(
            type="line",
            xref="x",
            yref="y",
            x0=0,
            y0=0.0416,  # Tasa libre de riesgo
            x1=risks[max_sharpe_idx],
            y1=returns[max_sharpe_idx],
            line=dict(
                color="rgba(0, 150, 136, 0.7)",
                width=2,
                dash="dash",
            ),
        )
    ],
    annotations=[
        dict(
            x=risks[max_sharpe_idx]/2,
            y=(returns[max_sharpe_idx] + 0.0416)/2,
            text="Línea de Mercado de Capitales",
            showarrow=True,
            arrowhead=2,
            arrowcolor="rgba(0, 150, 136, 0.7)",
            ax=40,
            ay=-40,
            font=dict(
                family="Arial, sans-serif",
                size=12,
                color="rgba(0, 150, 136, 1)"
            ),
        )
    ]
)

# Mostrar el gráfico
fig.show()

# --------------- GRÁFICO 2: COMPOSICIÓN DE CARTERAS MEJORADO ---------------
# Crear un gráfico de composición personalizado más estético
fig_comp = go.Figure()

# Obtener los pesos de todas las carteras
weights_all = np.array([portfolio.weights for portfolio in population_train])

# Obtener los nombres de los activos
asset_names = X.columns.tolist()

# Crear una paleta de colores más atractiva
colors = plt.cm.tab20(np.linspace(0, 1, len(asset_names)))
colors = [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})' for r, g, b, _ in colors]

# Ordenar por riesgo (eje x)
sort_indices = np.argsort(risks)
sorted_risks = risks[sort_indices]
sorted_weights = weights_all[sort_indices]

# Posiciones para el eje x
x = np.arange(len(sorted_risks))

# Agregar cada activo como una capa apilada
bottom = np.zeros(len(sorted_risks))
for i, asset in enumerate(asset_names):
    asset_weights = sorted_weights[:, i]
    fig_comp.add_trace(
        go.Bar(
            x=x,
            y=asset_weights,
            name=asset,
            marker_color=colors[i % len(colors)],
            hovertemplate=f"{asset}: %{{y:.2%}}<extra></extra>",
        )
    )

# Marcar la cartera con máximo Sharpe
max_sharpe_pos = np.where(sort_indices == max_sharpe_idx)[0][0]
fig_comp.add_trace(
    go.Scatter(
        x=[max_sharpe_pos],
        y=[1.05],
        mode="markers+text",
        marker=dict(
            symbol="triangle-down",
            color="#E50914",
            size=16
        ),
        text=["Máximo Sharpe"],
        textposition="top center",
        hoverinfo="none",
        showlegend=False
    )
)

# Configurar el diseño del gráfico
fig_comp.update_layout(
    title=dict(
        text="Composición de las Carteras de la Frontera Eficiente",
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    xaxis=dict(
        title="Carteras (ordenadas por riesgo creciente)",
        tickvals=[],
        showgrid=False
    ),
    yaxis=dict(
        title="Ponderación en la Cartera",
        tickformat=".0%",
        range=[0, 1.1],
        gridcolor='rgba(211, 211, 211, 0.6)'
    ),
    barmode='stack',
    plot_bgcolor='white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    )
)

# Mostrar el gráfico
fig_comp.show()

# --------------- GRÁFICO 3: RENDIMIENTO ACUMULADO MEJORADO ---------------
# Obtener los pesos de todas las carteras
weights_all = np.array([portfolio.weights for portfolio in population_train])

# Calcular los retornos diarios de todas las carteras usando X
retornos_diarios_all = X.dot(weights_all.T)

# Calcular la rentabilidad acumulada
cumulative_returns = (1 + retornos_diarios_all).cumprod(axis=0) - 1

# Crear el gráfico con plotly para mejor interactividad
fig_returns = go.Figure()

# Agregar todas las carteras en gris claro
for i in range(len(population_train)):
    if i != max_sharpe_idx:
        fig_returns.add_trace(
            go.Scatter(
                x=cumulative_returns.index,
                y=cumulative_returns.iloc[:, i],
                mode='lines',
                line=dict(color='rgba(200, 200, 200, 0.3)', width=1),
                showlegend=False,
                hoverinfo='skip'
            )
        )

# Agregar la cartera de máximo Sharpe en rojo brillante
fig_returns.add_trace(
    go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.iloc[:, max_sharpe_idx],
        mode='lines',
        line=dict(color='#E50914', width=3),
        name='Cartera de Máximo Sharpe',
        hovertemplate='<b>%{x}</b><br>Rendimiento: %{y:.2%}<extra></extra>'
    )
)

# Mejorar el diseño del gráfico
fig_returns.update_layout(
    title=dict(
        text='Rendimiento Acumulado de las Carteras en Período de Prueba',
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    xaxis=dict(
        title='Fecha',
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=False
    ),
    yaxis=dict(
        title='Rendimiento Acumulado',
        tickformat='.0%',
        gridcolor='rgba(211, 211, 211, 0.6)',
        zeroline=True,
        zerolinecolor='rgba(0, 0, 0, 0.2)',
        zerolinewidth=1
    ),
    plot_bgcolor='white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    legend=dict(
        y=0.99,
        x=0.01,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(211, 211, 211, 0.8)'
    ),
    hovermode='x unified'
)

# Resaltar la región donde la cartera de máximo Sharpe tiene mejor rendimiento
final_returns = cumulative_returns.iloc[-1, :]
max_final_return = final_returns.max()
max_final_return_idx = final_returns.argmax()

# Agregar anotación para destacar el mejor rendimiento
if max_final_return_idx == max_sharpe_idx:
    best_text = "¡La cartera de máximo Sharpe logró el mejor rendimiento!"
else:
    best_text = f"Cartera de máximo Sharpe: {final_returns[max_sharpe_idx]:.2%} de rendimiento"

fig_returns.add_annotation(
    x=cumulative_returns.index[-1],
    y=cumulative_returns.iloc[-1, max_sharpe_idx],
    text=best_text,
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowcolor="#E50914",
    ax=-150,
    ay=-40,
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="#E50914",
    font=dict(color="#E50914")
)

# Mostrar el gráfico
fig_returns.show()

# --------------- GRÁFICO 4: ESTADÍSTICAS DE LA CARTERA ÓPTIMA ---------------
# Crear un dashboard con las estadísticas clave de la cartera óptima
top_assets_idx = np.argsort(weights_all[max_sharpe_idx])[::-1][:10]  # Top 10 activos
top_assets = [asset_names[i] for i in top_assets_idx]
top_weights = [weights_all[max_sharpe_idx][i] for i in top_assets_idx]

# Crear un gráfico de pastel para los componentes principales
fig_stats = go.Figure()

fig_stats.add_trace(
    go.Pie(
        labels=top_assets,
        values=top_weights,
        hole=0.4,
        textinfo='label+percent',
        marker=dict(
            colors=plt.cm.Paired(np.linspace(0, 1, len(top_assets))),
            line=dict(color='white', width=2)
        ),
        textfont=dict(size=14),
        insidetextorientation='radial'
    )
)

# Configurar el diseño
fig_stats.update_layout(
    title=dict(
        text="Composición de la Cartera Óptima (Máximo Sharpe)",
        font=dict(size=24, family="Arial, sans-serif", color="#2F4F4F")
    ),
    annotations=[
        dict(
            text=f"Sharpe: {sharpe_ratios[max_sharpe_idx]:.3f}<br>Riesgo: {risks[max_sharpe_idx]:.2%}<br>Retorno: {returns[max_sharpe_idx]:.2%}",
            x=0.5,
            y=0.5,
            font=dict(size=16, family="Arial, sans-serif", color="#E50914"),
            showarrow=False
        )
    ],
    width=800,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    )
)

fig_stats.show()